Chapitre 7 - Du script à l'application - module, gestion des erreurs et recherche SQL
===

Notre application et nos compétences ont sérieusement grossies : nous dommes désormais capables de traiter du JSON, du CSV, de récupérer des informations depuis le web mais aussi de créer notre propre application web, bien qu'elle reste simple pour le moment.

Cette dite application web contient désormais 39 lignes de Python pour 2 pages et un type de ressource SQL. Vous vous rendrez vite compte que la manipulation de fichiers comme celui-ci rend les choses complexes. De fait, on préfère en général divisé les fichiers pythons comme nous divisons les templates : en les incluant ou en faisant appel à eux quand nous en avons besoin.

## Les modules et packages

Ce découpage en python s'appelle la modularisation : il s'agit de créer des modules qui à terme forme des packages (groupe de modules) et ainsi pouvoir se retrouver dans des applications un peu plus large que 39 lignes.

Nous avons déjà utilisés des modules. Ainsi quand on tapez

```python
from flask import Flask
```

Nous importions depuis le module principal du package `flask` la classe `Flask` qui nous permettait de créer notre application. Un module est en fait un fichier python. Ainsi, si je crée un fichier `modeles.py` pour y ranger l'ensemble de mes modèles de données SQL (et pouvoir m'y retrouver facilement), je pourrai à terme faire `from modeles import Place` !

Les modules peuvent comprendre tout ce que python peut faire, à savoir : 

- des variables
- des fonctions
- des classes

### Créer un package, c'est simple !

La création de package est en fait extrêment simple : 

0. On crée un fichier `run.py` qui nous permettra d'utiliser ce qui est dans gazetteer
1. On crée un dossier : `gazetteer/` par exemple.
2. On ajoute dans ce dossier un fichier `__init__.py`
3. On écrit dans ce fichier `__init__.py`.
4. On écrit dans ce fichier `ma_variable = 0`

Les fichiers `__init__.py` sont nécessaire pour que ce dossier soit compris comme package. Le résultat serait donc

```
gazetteer/
   |- __init__.py
run.py
```

Avec `__init__.py` qui comprend le code suivant :

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-01/gazetteer/__init__.py

Cela permettra à `run.py` d'utiliser la variable en faisant :

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-01/run.py

Et le résultat de son appel sera :

In [ ]:
%run cours-packages/exemple-01/run.py

#### Pour lancer l'exemple

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-01
python run.py
```

#### Résumé d'un module simple avec dossier

- Un module simple avec dossier comprenant un `__init__.py` est appelé ou importé en utilisant le nom du dossier. 
- On peut y stocker des variables, classes et fonctions.

### Un package, des modules

Il est possible pour un package de comporter plusieurs modules, ainsi pour notre Gazetteer il est possible de diviser par exemple les modeles des routes. Pour le moment, créons cet ensemble :

0. On crée un fichier `run.py` qui nous permettra d'utiliser ce qui est dans gazetteer
1. On crée un dossier : `gazetteer/` par exemple.
2. On ajoute dans ce dossier un fichier `__init__.py`
3. On crée un fichier `application.py`
4. On écrit dans ce fichier `mon_module = "application"`
5. On crée un fichier `modeles.py`
6. On écrit dans ce fichier `mon_module = "modeles"`

```
gazetteer/
   |- __init__.py
   |- application.py
   |- modeles.py
run.py
```

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/gazetteer/application.py

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/gazetteer/modeles.py

Cela permettra à `run.py` d'utiliser la variable en important les modules du packages. On importe des modules et sous-modules en rétablissant le chemin via des `.` :

```python
from gazetteer.application import mon_module
```

Ainsi, on écrira dans notre `run` :

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/run.py

Et le résultat de son appel sera :

In [ ]:
# Cette commande permet de lancer le script directement depuis ce module
! cd cours-packages/exemple-02 && python run.py

#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-02
python run.py
```

#### Problème ?

Nous rencontrons un premier problème : `mon_module` étant importé depuis deux modules (`gazetter.application` et `gazetteer.modeles`), les deux rentrent en conflit et la dernière prend le dessus. Nous avons plusieurs solutions à ce problème 

1. `from gazetteer import application` et `print(application.mon_module)` qui permet de garder le nom du sous-module
2. `import gazetteer.application` et `print(gazetteer.application.mon_module)` : long à écrire mais impossible d'avoir des conflits et compréhensible
3. `from gazetteer.application import mon_module as autre_nom` : on renomme, on perd l'information d'origine mais on reste sur un nom court.

Pour continuer l'exemple, on écrit un fichier `run2.py` qui reprend cette syntaxe :

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/run2.py

Et le résultat de son appel sera :

In [ ]:
# Cette commande permet de lancer le script directement depuis ce module
! cd cours-packages/exemple-02 && python run2.py

#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-02
python run2.py
```

#### Résumé d'un packages avec plusieurs modules

- Un package se construit avec un dossier comprenant un `__init__.py`
- Les fichiers qui le constituent sont importables comme modules (`from package import module`)
- Les imports de ressources dans des modules peuvent être construits de trois manière :
    - `import package.module`
        - Appelée via `package.module.ressource`
    - `from package import module`
        - Appelée via `module.ressource`
    - `from package.module import resource`
        - Appelée via `ressource`
- En cas de conflit, on peut:
    - changer de méthode d'import
    - renommer la ressource importée via `import ___ as ___` ou `from ___ import ___ as ____`

### De module à module : l'import relatif

Nous avons pour l'instant fait des imports uniquement depuis l'extérieur du package vers ce package. Il est cependant possible de faire ces imports de manière relative et il est d'ailleurs conseillé de le faire dans le cadre d'une application web.

Ces imports fonctionnent alors exactement comme les imports précédent mais en utilisant la syntaxe des chemins relatifs `..` et `.` où :

- `.` représente le dossier courrant
- `..` représente le dossier parent
- `...` représente le dossier grand-parent
- etc.

Ainsi, dans le cadre de nos modèles par exemple, il serait souhaitable de pouvoir diviser nos ressources entre les données utilisateurs et les données scientifiques pour se retrouver plus facilement dans les fichiers. Cela donnerait : 

```
gazetteer/
   |- __init__.py
   |- application.py
   |- modeles/
       |- __init__.py
       |- donnees.py
       |- utilisateurs.py
run.py
```

Si l'on a besoin d'une information d'application depuis `modeles.donnees`, on importera l'information via :

```python
from ..application import nom_dapplication
```

Si au contraire, on recherche une information liée à l'utilisateur, on écrira 

```python
from .utilisateurs import mon_utilisateur
```

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-03/gazetteer/application.py

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-03/gazetteer/modeles/utilisateurs.py

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-03/gazetteer/modeles/donnees.py

Et on importe simplement la fonction dans `gazetteer.modeles.donnees` depuis `run.py` :

In [ ]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-03/run.py

Et le résultat de son appel sera :

In [ ]:
# Cette commande permet de lancer le script directement depuis ce module
! cd cours-packages/exemple-03 && python run.py

#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-02
python run2.py
```

#### Résumé d'un packages avec plusieurs modules

- Un package peut avoir des sous-modules
- Les imports relatifs se font à l'aide des `.` :
    - `.` représente le dossier courrant
    - `..` représente le dossier parent
    - `...` représente le dossier grand-parent
    - etc.
- Depuis l'extérieur du package, on utilise l'import absolu

### Notre application en packages

Vous imaginez bien que si l'on peut diviser, il est recommandable de le faire. Et pour le faire, il vaut mieux pouvoir découper au mieux ce que vous pouvez faire. On recommande en général de découper :

1. Les modèles de données (et si possible les différents modèles de données : modèles scientifiques et modèles utilisateurs)
2. Les routes
3. La création de l'application

Regardez un peu le dossier `cours-packages/exemple-04`


#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-04
python run.py
```

#### Un petit bug ?

On nous annonce un problème avec les templates. Pour résoudre le problème, on spécifie le dossier ! En effet, `Flask()` prend deux paramètres nommés `templates` et `static` ! :

```python
app = Flask("Nom", template_folder="templates", static_folder="static") # Valeur par défaut ici
```

Mais en python, les dossiers relatifs sont pris depuis le fichier qui est exécuté. Hors, on exécute `run.py` et non `app.py`. Il faudrait donc écrire `./gazetteer/templates/` et `./gazetteer/static`. Mais si on change encore !

La technique vise à obtenir, via python, le chemin actuel de l'application. Il suffit pour cela d'exécuter le code suivant :

In [ ]:
% pycat cours-flask/exemple12/path.py
% run cours-flask/exemple12/path.py

##### Découpons le code !


- `import os` : on importe un package `os` permettant de faire des opérations liées au système
- `os.path.dirname()` : on récupère le nom de dossier du résultat de 
    - `os.path.abspath(__file__)` qui récupère le chemin absolu du dossier du fichier qui comprend ce code
        - **Attention:** `__file__` n'existe pas dans Jupyter Notebook
- `os.path.join()` permet de réaliser la jointure du chaîne avec les caractères de chemin de l'OS exécutant:
    - Sur Windows, il choisira `\`
    - Sur Linux et Mac, il choisira `/`
    
    
#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-flask/exemple12
python run.py
```

#### Attention !

On fera attention à importer les routes en fin de création d'application, sinon on aura une belle boucle d'import : 

1. `gazetteer.app` importe `gazetteer.routes` ligne 3
2. `gazetteer.routes` importe `gazetteer.app` ligne 2
3. `gazetteer.app` importe `gazetteer.routes` ligne 3
4. `gazetteer.routes` importe `gazetteer.app` ligne 2
5. `gazetteer.app` importe `gazetteer.routes` ligne 3
6. `gazetteer.routes` importe `gazetteer.app` ligne 2
7. *etc.*

## Les erreurs

On a souvent rencontré des erreurs. Et on en a même provoquées volontairement, par exemple, si on tentait de diviser d'obtenir un élément inexistant :

In [ ]:
liste = [0,1,2]
liste[3]

### Que faire quand on a une erreur

On peut éviter ignorer les erreurs en imbriquant un code dans un double bloc `try except` qui s'écrit comme un `if else` sauf que : 
- le bloc `except` est obligatoire si un `try` est ouvert
- le bloc `except` n'est executé que si une erreur apparaît

In [ ]:
try:
    print(liste[3])
except:
    print("Je me suis trompé !")
    print(liste[2])

### Que faire quand on veut gérer une erreur spécifique

Il est possible de vouloir gérer des erreurs de manière différente. Par exemple, dans le code suivant, nous avons une liste qui contient des dictionnaires, on peut avoir une erreur liée à l'index utilisé (IndexError) ou une erreur liée à une clée particulière. 

On fait alors suivre `except` par le nom ou les noms (séparés par des virgules) des erreurs que l'on veut gérer à part. Si une erreur ne fait pas partie des erreurs attendues, l'erreur n'est pas ignorée :

In [ ]:
objets = [
    {
        "nom": "R2D2",
        "prix": 85000
    },
    {
        "nom": "La Force"
    },
    {
        "nom": "BB8",
        "prix": 20000
    }
]

def convertir(index_objet, cour_credit_republicain=0.1):
    """ Convertit le prix d'un objet du monde de Star Wars en Trugut
    
    http://fr.starwars.wikia.com/wiki/Cr%C3%A9dit_Galactique_Standard
    """
    try:
        print("{objet} a un prix de {credits} soit {trugut} Truguts".format(
            objet=objets[index_objet]["nom"],
            credits=objets[index_objet]["prix"],
            trugut=objets[index_objet]["prix"]/cour_credit_republicain
        ))
        return objets[index_objet]["prix"]/cour_credit_republicain
    except IndexError:
        print("`Ce qui n'est pas dans nos collections n'existe pas`")
        return None
    except KeyError:
        print("L'objet {} n'a pas toutes les données nécessaires".format(index_objet))
        return None

convertir(0)
print("---")
convertir(1)
print("---")
convertir(3)
print("---")
convertir(2, 0)


### Récupérer l'erreur et la relancer

Il peut être intéressant de récupérer l'erreur et de la relance : imaginons que nous avons un bloc de gestion de données et qu'une erreur arrive après avoir déjà traîté un gros nombre de données. On pourrait alors sauvegarder ce que l'on a, puis lancer l'erreur pour faire du debuggage.

In [ ]:
try:
    print(liste[3])
except Exception as ma_variable_erreur:
    print("Je me suis trompé !")
    raise ma_variable_erreur

#### Lecture de code :

- On ajoute Exception qui permet de cibler toutes les erreurs. Cela pourrait être une erreur spécifique
    - On stocke cette erreur via `as nom_de_variable`
- On fait toutes les opérations que l'on veut
- On utilise ensuite `raise` avec l'erreur à lancer

### Important

La gestion d'erreur ne devrait être utilisée qu'en cas de force majeur ! Elle est en effet plus consommatrice que des simples `if`-`else` : tant que vous pouvez le prévoir, tentez de couvrir votre code via des conditions et non des `try-except`

### Exercice

Le code suivant peut créer une erreur :

```python
lieux = {
    0: {
        "nom": "Col. Lugdunum",
        "moderne": "Lyon",
        "latlong": [45.762095775, 4.822438025],
        "type": "ville",
        "description": "Col. Lugdunum was a Roman military colony from 43 BC and a major center in Gaul. Marcus "
                       "Agrippa was involved in its expansion and two Roman emperors, Claudius and Caracalla, "
                       "were born there."
    },
    1: {
        "nom": "Samarobriva Ambianorum",
        "moderne": "Amiens",
        "type": "ville",
        "description": "An ancient place, cited: BAtlas 11 C3 Samarobriva Ambianorum ",
        "latlong": [49.8936075, 2.297948]
    }
}
@app.route("/place/<int:place_id>")
def lieu(place_id):
    return render_template("pages/place.html", nom="Gazetteer", lieu=lieux[place_id])
```

1. Pouvez-vous dire dans quelles conditions ?
2. Récupérez le nom de l'erreur
3. Ajoutez un `try-except`
    4. Quel code HTTP devriez-vous rajouter ?